In [3]:
include("../LiPoSID.jl")

using LinearAlgebra

using DynamicPolynomials

using SymPy

using Plots

using Dates

using HDF5

data_dir = "../DATA/NO_CUT/"

γ_list = [ "0.079477",  "0.25133", "0.79477", "2.5133", "7.9477", "25.133",  "79.477", "251.33"]

γ_list_disp = [ "0.0795",  "0.2513", "0.7947", "2.5133", "7.9477", "25.133",  "79.477", "251.33"];

b⁰ = [0, 0, 1, 1]
b¹ = [0, 0, -1, 1]
bˣ = [1, 0, 0, 1]
bʸ = [1, 1, 0, 1]

function read_as_operators(models_file_name, γ)
    h5open(models_file_name, "r") do file
        w = read(file["gamma_"*string(γ)]["w"])
        H = [ w        0
              0        0   ]
        
        g = read(file["gamma_"*string(γ)]["g"])
        J = [ 0     sqrt(abs(g))
              0     0. + 0im  ]

        return H, J
    end
end


read_as_operators (generic function with 1 method)

In [4]:
using QuantumOptics
basis = NLevelBasis(2)

test_files = ["State_D"*string(n) for n=1:20];

#models_file_name= "../MODELS/DMD_TS_POP_w&g_bx_2024-Jan-02_at_15-50.h5"
#models_file_name= "../MODELS/DMD_TS_POP_w&g_bx_2024-Jan-05_at_13-18.h5"
models_file_name= "../MODELS/DMD_TS_POP_RND_w&g2_pd_bx_2024-Jan-31_at_16-14.h5"

"../MODELS/DMD_TS_POP_RND_w&g2_pd_bx_2024-Jan-31_at_16-14.h5"

In [5]:
date_and_time_string =  string(Dates.format(now(), "yyyy-u-dd_at_HH-MM"))

tests_dir = "../TESTS/"
tests_data_file_name = "DMD_TS_POP_RND_testD20_"*date_and_time_string * ".h5"

for γᵢ in γ_list

    println()

    println("γ="*γᵢ)

    h5open(tests_dir*tests_data_file_name,"cw") do fid
        γ_group = create_group(fid, "gamma_"*γᵢ)
    end 


    for df in test_files # loop over initial states
        
        print(df*" - ")

        ρₛ, tₛ = LiPoSID.get_rho_series(data_dir*df*"_CUT_data.h5", γᵢ) #"_2CUT_data.h5"

        ρₛ = convert(Vector{Matrix{ComplexF64}}, ρₛ)
        t = convert(Vector{Float64}, tₛ)

        ρᵗˢᵗ = [DenseOperator(basis,Hermitian(ρₜ)) for ρₜ in ρₛ]

        Hˢⁱᵈₗₘₑ, Jˢⁱᵈₗₘₑ = read_as_operators(models_file_name, γᵢ)

        tˢⁱᵈₗₘ, ρˢⁱᵈₗₘₑ  = timeevolution.master(t, ρᵗˢᵗ[1], DenseOperator(basis,Hˢⁱᵈₗₘₑ), [DenseOperator(basis,Jˢⁱᵈₗₘₑ)])

        Fˢⁱᵈₗₘₑ = [abs(fidelity(ρ₁, ρ₂)) for (ρ₁, ρ₂) in zip(ρᵗˢᵗ, ρˢⁱᵈₗₘₑ)]

        h5open(tests_dir*tests_data_file_name,"cw") do fid
            γ_group = open_group(fid, "gamma_"*γᵢ) # open coupling group
            init_state_group = create_group(γ_group, df) # create initial state group
            init_state_group["F_sid_lme"] = convert.(Float64, Fˢⁱᵈₗₘₑ)
        end
                    
        print(minimum(Fˢⁱᵈₗₘₑ))
        print(" | ")
    
    end

    println()

end

println()
println(tests_data_file_name)
    


γ=0.079477
State_D1 - 

0.9927608346373464 | State_D2 - 0.99276539227432 | State_D3 - 0.9703169450090892 | State_D4 - 0.9703245902600357 | State_D5 - 

0.9888491647723223 | State_D6 - 0.988849179893852 | State_D7 - 0.9821680484176939 | State_D8 - 0.9821732387629906

 | State_D9 - 0.9987413248736724 | State_D10 - 0.9622425604524334 | State_D11 - 0.9703169333211462 | State_D12 - 0.970324602885806

 | State_D13 - 0.9927608565297558 | State_D14 - 0.992765402507007 | State_D15 - 

...

 | State_D13 - 0.8656184484478534 | State_D14 - 0.8461595698310158 | State_D15 - 0.7550158111159935 | State_D16 - 0.7550158111159928 | State_D17 - 

0.9540241857567872 | State_D18 - 0.9402632541580447 | State_D19 - 0.7989450699841589 | State_D20 - 0.817691465232838 | 

γ=251.33
State_D1 - 0.8050840728270088 | State_D2 - 0.7679939547691821 | State_D3 - 

0.7831670269441855 | State_D4 - 0.7491213653781301 | State_D5 - 0.6414808384103929 | State_D6 - 0.6414808384103934 | State_D7 - 0.9694307290338546 | State_D8 - 0.9439340233855373 | State_D9 - 0.7104003788154045 | State_D10 - 0.6804636883374737 | State_D11 - 

0.7831670269441855 | State_D12 - 0.7491213653781301 | State_D13 - 0.8050840728270088 | State_D14 - 0.7679939547691821 | State_D15 - 0.6303284132430091 | State_D16 - 0.6303284132430091 | State_D17 - 0.9694307290338546 | State_D18 - 0.9439340233855373

 | State_D19 - 0.6804636883374737 | State_D20 - 0.7104003788154045 | 

DMD_TS_POP_RND_testD20_2024-Jan-31_at_16-18.h5
